In [367]:
import json
import numpy as np
import pandas as pd

import MeCab

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.manifold import TSNE

import plotly.express as px
from datetime import datetime


In [368]:
stories_json = []
with open('./output_stories_0123.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

839
dict_keys(['index', 'stories_index', 'title', 'broadcasting_date', 'abstracts_list'])


In [369]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママにた...
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、天...


In [370]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 品詞を絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

In [371]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_joined'].str.len() > 0] 
sentences = stories_df["abstract_joined"].apply(mecab_tokenizer)
print(sentences)

0      [何, のび太, 魔法, 魔法, 便利, ちがい, もしもボックス, 現実, 世界, 魔法,...
1      [朝, 寝坊, のび太, ドラえもん, 爆発, コショウ, コショウ, 行き先, そば, 人...
2      [みんな, 家族, お花見, 話, しかた, のび太, ママ, パパ, ゴルフ, 予定, の...
3      [スネ夫, ハワイ旅行, じまん, のび太, 自分, ハワイ, ママ, 給料日, 前, お金...
4      [学校, 天女, 羽衣, 羽子, ろ, 劇, げき, こと, のび太, しずか, 劇, 最後...
                             ...                        
834    [星がきれい, 夜, のび太, スネ夫, 家, 前, 庭, 星空, パーティー, 女の子, ...
835    [おなか, 食事, のび太, ご飯, ありがたみ, ママ, のび太, まじめ, 米, パン,...
836    [子供達, 鬼ごっこ, の, のび太, 自分たち, 野郎, みんな, ジャイアン, スネ夫,...
837    [５, 回, 連続, ０, 点, のび太, 来週, テスト, ０, 点, ママ, 鬼, 顔,...
838    [冬, スポーツ, ウインタースポーツ, ランド, のび太, たち, スキー, スケート, ...
Name: abstract_joined, Length: 839, dtype: object


In [372]:
abstract_list = sentences.tolist()
trainings = [TaggedDocument(words = data, tags = [i]) for i,data in enumerate(abstract_list)]
m = Doc2Vec(documents= trainings, dm = 1, window=8, min_count=10, workers=4)


In [373]:
vector_list = m.dv.vectors
tsne = TSNE(n_components=2)
vector_enbedded = tsne.fit_transform(vector_list)
print(vector_enbedded)

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[[-18.738823    9.606904 ]
 [  2.5593333   9.504277 ]
 [-17.893543   -7.303797 ]
 ...
 [-22.133114  -25.026165 ]
 [ -9.794731  -24.665638 ]
 [-19.76661   -25.078876 ]]


In [374]:
vector_df = pd.DataFrame.from_dict({
    'vector': [v for v in vector_list],
    'vector_x': [v[0] for v in vector_enbedded],
    'vector_y': [v[1] for v in vector_enbedded]
})
stories_df_merged = stories_df.join(vector_df)
stories_df_merged['broadcasting_dt'] = stories_df_merged['broadcasting_date'].apply(lambda d: datetime.strptime(d, '%Y年%m月%d日'))
stories_df_merged['broadcasting_year'] = stories_df_merged['broadcasting_dt'].dt.year

In [375]:
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', color='broadcasting_year', hover_data=["index", "title"])
fig.show()

In [376]:
df = px.data.tips()
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', hover_data=["index", "title"], facet_row="broadcasting_year", width=800, height=2000)
fig.show()

In [377]:
sampled_df = stories_df_merged.sample(1)
print(sampled_df.head())
sampled_df = sampled_df.rename(columns={"vector": "sampled_vec"})[["sampled_vec"]]


    index  stories_index     title broadcasting_date  \
522  0586              1  「ネンドロイド」        2020年1月18日   

                                        abstracts_list  \
522  [部屋に入ったのび太は、おし入れをゴソゴソしているナゾの物体を見つける。その物体がドラえもん...   

                                       abstract_joined  \
522  部屋に入ったのび太は、おし入れをゴソゴソしているナゾの物体を見つける。その物体がドラえもんの...   

                                                vector   vector_x  vector_y  \
522  [-0.042228714, 0.03145719, -0.009989154, -0.00... -10.256961  8.571959   

    broadcasting_dt  broadcasting_year  
522      2020-01-18               2020  


In [378]:
stories_df_merged = stories_df_merged.assign(key=1).merge(sampled_df.assign(key=1), on='key').drop('key', axis=1)
stories_df_merged

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined,vector,vector_x,vector_y,broadcasting_dt,broadcasting_year,sampled_vec
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...,"[-0.05149672, 0.03636912, -0.02340619, 0.00349...",-18.738823,9.606904,2013-03-15,2013,"[-0.042228714, 0.03145719, -0.009989154, -0.00..."
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...,"[-0.03853727, 0.024850013, -0.016734624, 0.001...",2.559333,9.504277,2013-04-12,2013,"[-0.042228714, 0.03145719, -0.009989154, -0.00..."
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママにた...,"[-0.05434031, 0.034021407, 0.0009172482, -0.00...",-17.893543,-7.303797,2013-04-12,2013,"[-0.042228714, 0.03145719, -0.009989154, -0.00..."
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...,"[-0.017568838, 0.016716748, -0.0040159654, 0.0...",22.053534,10.526115,2013-04-26,2013,"[-0.042228714, 0.03145719, -0.009989154, -0.00..."
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、天...,"[-0.02971179, 0.014156782, -0.0072400025, -0.0...",13.678095,20.793144,2013-04-26,2013,"[-0.042228714, 0.03145719, -0.009989154, -0.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...
834,0741,1,「夜空に輝くピザ・ギョーザ」,2023年1月14日,[星がきれいにかがやく夜、のび太がスネ夫の家の前を通ると、庭で星空パーティーが開かれていた。...,星がきれいにかがやく夜、のび太がスネ夫の家の前を通ると、庭で星空パーティーが開かれていた。女...,"[-0.06655661, 0.06380969, -0.026278881, -0.004...",-23.469191,-24.188925,2023-01-14,2023,"[-0.042228714, 0.03145719, -0.009989154, -0.00..."
835,0742,0,「ありがたみわかり機」,2023年1月21日,[おなかがいっぱいだからと食事を残すのび太に、ご飯が食べられるありがたみについて話すママ。と...,おなかがいっぱいだからと食事を残すのび太に、ご飯が食べられるありがたみについて話すママ。とこ...,"[-0.020004068, 0.015646802, -0.0015943793, 0.0...",19.063536,18.134411,2023-01-21,2023,"[-0.042228714, 0.03145719, -0.009989154, -0.00..."
836,0742,1,「のび泥棒をタイホせよ！」,2023年1月21日,[小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろうとみんな...,小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろうとみんなを...,"[-0.070890315, 0.061606534, -0.020942321, 0.00...",-22.133114,-25.026165,2023-01-21,2023,"[-0.042228714, 0.03145719, -0.009989154, -0.00..."
837,0743,0,「鬼のパンツはいいパンツ？」,2023年1月28日,[５回連続で０点を取ってしまったのび太。来週のテストも０点だったら、ママが鬼（おに）になって...,５回連続で０点を取ってしまったのび太。来週のテストも０点だったら、ママが鬼（おに）になってし...,"[-0.04403009, 0.038832165, -0.020036262, -0.02...",-9.794731,-24.665638,2023-01-28,2023,"[-0.042228714, 0.03145719, -0.009989154, -0.00..."


In [379]:
from scipy.spatial.distance import cosine

stories_df_merged["cosine_similarity"] =  stories_df_merged.apply(lambda v: 1 - cosine(v["vector"], v["sampled_vec"]), axis=1)
stories_df_merged_sort = stories_df_merged[["title", "abstract_joined", "cosine_similarity"]].sort_values("cosine_similarity", ascending=False)


In [380]:
stories_df_merged_sort

,title,abstract_joined,cosine_similarity
522,「ネンドロイド」,部屋に入ったのび太は、おし入れをゴソゴソしているナゾの物体を見つける。その物体がドラえもんの...,1.000000
716,「のび太とのび太」,ママにたのまれ、庭で草むしりをするのび太。終わったら買い物をお願いと声をかけるママだったが、...,0.984356
23,「なんでもアイス棒」,本場イタリアから取り寄せたという最高級ジェラートアイスをスネ夫にじまんされたのび太は、つい...,0.984276
393,「地平線テープ」,地平線を「チダイラ線」と読んでしまい、ママに怒られるのび太。本物の地平線を見たことがないのび...,0.983666
123,「おすそわけガム」,\r\n 最近、おやつが少ないことをなげくのび太とドラえもん。道で会ったスネ夫の家に遊びに行...,0.983520
...,...,...,...
814,「ななころびてんとう虫」,お使いのとちゅうできげんの悪そうなジャイアンを見かけ、別の道に行こうとするのび太だったが、空...,-0.899703
206,「脱出!!巨大クリスマスケーキ」,\r\n クリスマス会に参加するため、のび太がめずらしくおしゃれをしていると、ドラミがクリス...,-0.920172
314,「テレパしい」,自分が何をして欲しいか、寝ているだけでもわかってもらえる道具はないかと言い出すのび太。あきれ...,-0.943037
149,「わすれろ草」,空き地にジャイアンとスネ夫の似顔絵を落書きし、みんなに見せびらかしていたのび太。するとそこ...,-0.944726
